In [40]:
import configparser
import json
import os
import pandas as pd
import requests
import sqlite3
import datetime
from bs4 import BeautifulSoup
from tabulate import tabulate

# 로그 기록 시작 함수
def log_started():
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write("\n" + "="*50 + "\n")
        timestamp = datetime.datetime.now().strftime('%Y-%B-%d-%H-%M-%S')
        log_file.write(f"New execution at {timestamp}")
        log_file.write("\n" + "="*50 + "\n")

# 로그 기록 함수
def log_message(message, level="INFO"):
    timestamp = datetime.datetime.now().strftime('%Y-%B-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp} - {level} - {message}\n")

# 설정 파일 읽기
def load_config(config_path='config.ini'):
    if not os.path.exists(config_path):
        log_message(f"Configuration file '{config_path}' not found.", level="ERROR")
        raise FileNotFoundError((f"Configuration file '{config_path}' not found."))
    
    config = configparser.ConfigParser()
    config.read(config_path)
    
    if 'DEFAULT' not in config or 'URL' not in config['DEFAULT'] or 'TABLE_CLASS' not in config['DEFAULT']:
        log_message("Invalid or missing configuration values in 'config.ini'.", level="ERROR")
        raise ValueError("Invalid or missing configuration values in 'config.ini'.")
    
    return config['DEFAULT']['URL'], config['DEFAULT']['TABLE_CLASS']
    

# Save
def save_gdp_data(df, output_csv_file='extracted_gdp_data.csv', output_json_file='extracted_gdp_data.json'):
    log_message("Saving Extracted Data")
    try:
        df.to_csv(output_csv_file, index=False)
        df.to_json(output_json_file, orient='records', force_ascii=False, indent=4)
        log_message(f"Data saved: CSV ({output_csv_file}), JSON ({output_json_file})")
    except Exception as e:
        log_message(f"Failed to save data: {str(e)}", level="ERROR")
        raise
    
def extract_gdp_data(url, table_class):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() # HTTP 응답 상태 코드를 확인. 200번대가 아닌 경우(예: 404, 500 등) HTTPError 예외를 발생시킴
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': table_class})
        
        if table is None:
            log_message("No table found with the specified class.", level="ERROR")
            raise ValueError("Failed to locate the GDP table on the webpage.")
            
        df = pd.read_html(str(table))[0]  # 위키피디아에서 제공하는 표를 Pandas로 읽고 객체를 문자열로 변환
        
        df = df.iloc[:, [0, 1, 2]]  # 필요한 칼럼만 선택 (모든 행과 0, 1, 2번째 열을 선택)
        df.columns = ['Country', 'GDP (Nominal)', 'Year']
        
        df = df.dropna(subset=['Country', 'GDP (Nominal)']) # NaN 데이터 제거 
        df['GDP (B USD)'] = ( # GDP 값 정리 및 변환
            df['GDP (Nominal)']
            .str.replace(r'[^\d.]', '', regex=True)  # 숫자와 소수점 이외 제거
            .replace('', '0')  # 빈 문자열을 '0'으로 대체
            .astype(float)  # float으로 변환
            / 1e3  # 단위를 B USD로 변환
        )
        df['Year'] = df['Year'].str.replace(r'\[.*?\]', '', regex=True) # 각주 제거 (sup 이런 게 자꾸 따라와서..)
        df = df[['Country', 'GDP (B USD)', 'Year']]
        
        return df
        
    except Exception as e:
        log_message(f"Error during data extraction: {str(e)}", level="ERROR")
        raise


# Transform
def transform_gdp_data(df):
    log_message("Starting Data Transmission")
    try:
        df = df.sort_values(by='GDP (B USD)', ascending=False)  # 정렬
        df['GDP (B USD)'] = df['GDP (B USD)'].round(2)  # 소수점 2자리로 반올림
        
        with open('country_region_table.json', 'r', encoding='utf-8') as region_file:
            region_data = json.load(region_file)
        df['Region'] = df['Country'].map(region_data)
        return df
        
    except Exception as e:
        log_message(f"Error during data transformation: {str(e)}", level="ERROR")
        raise



def load_gdp_data(df):
    log_message("Loading data into SQLite database")
    try:
        # SQLite 데이터베이스에 연결
        conn = sqlite3.connect('World_Economies.db')
        
        df[['Country', 'GDP (B USD)', 'Year', 'Region']].rename(
            columns={'GDP (B USD)': 'GDP_USD_billion'}
        ).to_sql( # 데이터프레임 데이터를 SQL 테이블로 변환하여 데이터베이스에 저장하는 pandas 메서드입
            'Countries_by_GDP', conn, if_exists='replace', index=False
        )
        
        conn.close()
        log_message("Data successfully loaded into SQLite database")
        
        
    except Exception as e:
        log_message(f"Error while loading data into SQLite database: {str(e)}", level="ERROR")
        raise



# GDP가 100B USD 이상인 국가 필터링
def filtered_100USD(df):
    filtered_100 = df[df['GDP (B USD)'] >= 100]
    print("Countries with a GDP of over 100B USD")
    print(filtered_100)
    return filtered_100


# Region별 상위 5개 국가의 GDP 평균 계산
def region_top5_calculate(df):
    region_top5_avg = (
        df.groupby('Region')
        .apply(lambda x: x.nlargest(5, 'GDP (B USD)')['GDP (B USD)'].mean())
        .reset_index(name='Top 5 Avg GDP (B USD)')
    )
    print("Average GDP of top 5 countries by region")
    print(region_top5_avg)
    return region_top5_avg


# 추가 요구사항 구현
def display_countries_with_gdp_over_100():
    log_message("Displaying countries with GDP over 100B USD")
    try:
        conn = sqlite3.connect('World_Economies.db')
        query = "SELECT Country, GDP_USD_billion FROM Countries_by_GDP WHERE GDP_USD_billion >= 100"
        result = pd.read_sql_query(query, conn)
        conn.close()
        
        print("Countries with GDP >= 100B USD:")
        print(tabulate(result, headers='keys', tablefmt='fancy', showindex=False))
        
    except Exception as e:
        log_message(f"Error querying database for countries with GDP >= 100B: {str(e)}", level="ERROR")
        raise
    
# Region별 상위 5개 국가의 GDP 평균 계산 및 출력
def display_region_top5_average_gdp():
    log_message("Calculating average GDP of top 5 countries by region")
    try:
        conn = sqlite3.connect('World_Economies.db')
        query = """
        WITH RankedCountries AS (
            SELECT Country, GDP_USD_billion, Region,
                   RANK() OVER (PARTITION BY Region ORDER BY GDP_USD_billion DESC) AS Rank
            FROM Countries_by_GDP
            WHERE Region IS NOT NULL
        )
        SELECT Region, AVG(GDP_USD_billion) AS Avg_Top5_GDP
        FROM RankedCountries
        WHERE Rank <= 5
        GROUP BY Region
        """
        result = pd.read_sql_query(query, conn)
        conn.close()
        
        print("Average GDP of top 5 countries by region (excluding None):")
        print(tabulate(result, headers='keys', tablefmt='pretty', showindex=False))
        
    except Exception as e:
        log_message(f"Error querying database for top 5 average GDP: {str(e)}", level="ERROR")
        raise


def etl_process():
    try:
        log_started()
        log_message("ETL Process Started")
        
        # 설정 로드
        url, table_class = load_config()
        
        # Extract
        extracted_data = extract_gdp_data(url, table_class)
        
        # Save Extracted Data
        save_gdp_data(extracted_data)
        
        # Transform
        transformed_data = transform_gdp_data(extracted_data)
        
        # Save Transformed Data
        save_gdp_data(transformed_data, 'transformed_gdp_data.csv', 'transformed_gdp_data.json')
        
        # Load into SQLite Database
        load_gdp_data(transformed_data)


        # Additional Analyses
        # display_countries_with_gdp_over_100()
        display_region_top5_average_gdp()


        # # 추가요구사항 전 출력과정
        # filtered_data = filtered_100USD(transformed_data)
        # region_top5_data = region_top5_calculate(transformed_data)
        
        log_message("ETL Process Completed Successfully")        
    except Exception as e:
        log_message(f"ETL Process is Failed: {str(e)}", level="ERROR")
        
        
if __name__ == "__main__":
    etl_process()

Average GDP of top 5 countries by region (excluding None):
+---------------+-------------------+
|    Region     |   Avg_Top5_GDP    |
+---------------+-------------------+
|    Africa     |      256.134      |
|     Asia      | 6255.969999999999 |
|    Europe     |     3318.112      |
| North America |      6946.5       |
|    Oceania    | 734.8399999999999 |
| South America |      797.566      |
+---------------+-------------------+


/var/folders/rd/d9knhb6x6nj7rd817gqlcsr00000gn/T/ipykernel_17505/3329315340.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # 위키피디아에서 제공하는 표를 Pandas로 읽고 객체를 문자열로 변환


In [29]:
df = pd.DataFrame( {
    '2020': [2500, 1500, 3000],
    '2021': [2700, 1800, 3600],
    '2022': [3000, 2400, 4200],
    '2023': [3500, 3000, 5600] 
    })
print(df)
print('-' * 50)

df.columns = ['1', '2', '3', '4']
print(df)
print('-' * 50)

filtered = df[df['1'] >= 3000]
print(filtered)

   2020  2021  2022  2023
0  2500  2700  3000  3500
1  1500  1800  2400  3000
2  3000  3600  4200  5600
--------------------------------------------------
      1     2     3     4
0  2500  2700  3000  3500
1  1500  1800  2400  3000
2  3000  3600  4200  5600
--------------------------------------------------
      1     2     3     4
2  3000  3600  4200  5600
